# Syntax Tree Parsing using CKY

## Name: Kevin Veeder

For this project, I am building a simple parser trained on the ATIS (Air Traffic Information System) portion of the Penn Treebank. The training data consists of short queries and commands spoken by users interacting with a simulated robot travel agent.

The key files involved and their roles are:

- `train.trees.pre.unk`: Contains the training data trees — binary branching trees where all words appearing only once are replaced with <unk>.

- `dev.strings`: Contains the strings for the development data set.



## What is CKY Parsing?

CKY Parsing, named after its inventors John Cocke, Daniel Younger, and Tadao Kasami, is a widely-used algorithm for parsing sentences based on context-free grammars. Its efficiency and ability to handle ambiguous grammatical structures make it a fundamental technique in natural language processing.

## Importance in NLP

Syntactic parsing — analyzing a sentence’s structure using formal grammar rules — is essential for understanding language. The CKY algorithm uses a bottom-up parsing approach to break down complex sentences into constituent parts, supporting tasks like machine translation, sentiment analysis, and information extraction. By implementing CKY Parsing, I am gaining insight into how computers process and interpret human language, a critical skill in NLP.

## 01 - Setup

To begin, I’m setting up the environment for the CKY Parsing process. This involves importing the necessary libraries and preparing everything needed to perform parsing.

First, I import the required libraries and methods (ensuring they are installed beforehand):

    - NLTK
    - Tree from the nltk.tree
    - ViterbiParser from nltk.parse
    - svgling
    - math

In [1]:
# Run these pip commands if you don't have these libraries installed already
# !pip install nltk
# !pip install svgling
# !pip install dill

In [2]:
import nltk
from nltk.tree import Tree
from nltk.parse import ViterbiParser
import svgling
import math

## 02 - Importing and Visualizing the Syntax Tree

### What is a Syntax Tree?

A syntax tree, also known as a parse tree, is a tree representation of the syntactic structure of a sentence. In linguistics and NLP, these trees are crucial for understanding how different parts of a sentence relate to each other according to the rules of grammar.

### Importance of Syntax Trees in NLP

Syntax trees play a vital role in various NLP applications. They help in:

- **Understanding Sentence Structure:** By breaking down a sentence into its constituent parts, syntax trees make it easier to analyze the grammatical structure.
- **Improving Language Models:** They are essential in developing more accurate models for tasks like machine translation, speech recognition, and sentiment analysis.
- **Information Extraction:** Syntax trees aid in extracting meaningful information from complex sentences, which is crucial in tasks like question answering and text summarization.

### Importing and Visualizing the Syntax Tree


I start by importing a syntax tree from the provided file `train.trees.pre.unk` to visualize its structure. This helps deepen my understanding of sentence structure in NLP and the role syntax trees play in this process.

1. Read the First Line of the File: Using Python’s built-in `open` function, I access the file and read the first line, which contains the syntax tree of a sentence in a specific format. The command `first_line = file.readline().strip()` reads the line and removes any leading or trailing whitespace.

2. Parse the Syntax Tree: Next, I parse the string to create a syntax tree by applying the `Tree.fromstring(first_line)` method from NLTK. This converts the string representation into a Tree object, which I store in the variable `first_tree`.

3. Print the Partial Syntax Tree: Finally, I visualize the tree using the `pretty_print` method on the `first_tree object`.

In [3]:
with open("train.trees.pre.unk") as file:
    first_line = file.readline().strip()

first_tree = Tree.fromstring(first_line)
print(first_tree.pretty_print())

                                        TOP                                                         
                 ________________________|_______________________________________________________    
               S_VP                                                                              |  
  ______________|________________________                                                        |   
 |                                       NP                                                      |  
 |         ______________________________|_________                                              |   
 |        |                                       NP*                                            |  
 |        |                 _______________________|_______                                      |   
 |        |                |                              NP*                                    |  
 |        |                |                  _____________|___________                

## 03 - Convert Trees to Chomsky Normal Form (CNF) and Extract Productions

In this stage of the project, I transform the syntax trees into Chomsky Normal Form (CNF) and extract grammatical productions from them. This transformation is an essential step in building a grammar that can be used for parsing.

### What is Chomsky Normal Form?

Chomsky Normal Form is a way of rewriting and simplifying the grammar of a language. A grammar in CNF has its production rules restricted to a certain standard form. Specifically, each rule must be either:
- A -> BC, where 'A', 'B', and 'C' are non-terminal symbols, and 'B' and 'C' are not the start symbol.
- A -> a, where 'A' is a non-terminal symbol and 'a' is a terminal symbol.
This form is particularly useful for parsing algorithms and simplifying the analysis of grammatical structures.

### Convert Trees to Chomsky Normal Form (CNF) and Extract Productions

1. Read Syntax Trees from File:
I begin by reading all syntax trees from the `train.trees.pre.unk` file into a list, using the same approach as in the earlier step.

2. Convert Trees to CNF:
Each tree is then converted into Chomsky Normal Form using the `Tree.chomsky_normal_form` method from the NLTK library, with the parameter `horzMarkov=2` to control binarization.

3. Extract Productions:
After converting the trees to CNF, I extract the grammatical productions from them. These production rules define how sentences are constructed and are fundamental for building a formal grammar.

I create a list called `productions` to collect all the rules. For each CNF tree, I use the `productions()` method to extract its grammatical rules and append them to the `productions` list. This collection of rules will later be used to construct the grammar for parsing.


In [4]:
with open("train.trees.pre.unk") as file:
    lines = file.readlines()
    lines = [line.rstrip('\n') for line in lines]

trees_list = []
for line in lines:
    trees = Tree.fromstring(line)
    trees_list.append(trees)

In [5]:
for trees in trees_list:
    trees = Tree.chomsky_normal_form(trees, horzMarkov=2)

In [6]:
productions = []
for trees in trees_list:
    rules = trees.productions()
    for rule in rules:
        productions.append(rule)

productions

[TOP -> S_VP PUNC,
 S_VP -> VB NP,
 VB -> 'List',
 NP -> NP NP*,
 NP -> DT NNS,
 DT -> 'the',
 NNS -> 'flights',
 NP* -> PP NP*,
 PP -> IN NP_NNP,
 IN -> 'from',
 NP_NNP -> 'Baltimore',
 NP* -> PP SBAR,
 PP -> TO NP_NNP,
 TO -> 'to',
 NP_NNP -> 'Seattle',
 SBAR -> WHNP_WDT S_VP,
 WHNP_WDT -> 'that',
 S_VP -> VBP PP,
 VBP -> 'stop',
 PP -> IN NP_NNP,
 IN -> 'in',
 NP_NNP -> 'Minneapolis',
 PUNC -> '.',
 TOP -> SQ PUNC,
 SQ -> VBZ SQ*,
 VBZ -> 'Does',
 SQ* -> NP VP,
 NP -> DT NN,
 DT -> 'this',
 NN -> 'flight',
 VP -> VB NP_NN,
 VB -> 'serve',
 NP_NN -> 'dinner',
 PUNC -> '?',
 TOP -> S PUNC,
 S -> NP_PRP VP,
 NP_PRP -> 'I',
 VP -> VBP NP,
 VBP -> 'need',
 NP -> NP NP*,
 NP -> DT NN,
 DT -> 'a',
 NN -> 'flight',
 NP* -> PP NP*,
 PP -> TO NP_NNP,
 TO -> 'to',
 NP_NNP -> 'Seattle',
 NP* -> VP VP,
 VP -> VBG PP,
 VBG -> 'leaving',
 PP -> IN NP_NNP,
 IN -> 'from',
 NP_NNP -> 'Baltimore',
 VP -> VBG NP,
 VBG -> '<unk>',
 NP -> NP PP,
 NP -> DT NN,
 DT -> 'a',
 NN -> 'stop',
 PP -> IN NP_NNP,


## 04 - Inducing a Probabilistic Context-Free Grammar (PCFG) from Productions

After extracting the productions from the CNF trees, the next critical step is to induce a Probabilistic Context-Free Grammar (PCFG) from those productions. This is a key component of syntactic parsing in NLP.

### What is a Probabilistic Context-Free Grammar (PCFG)?

A PCFG extends the standard context-free grammar by associating probabilities with each production rule. These probabilities reflect how often each rule is used in the language data, allowing the grammar to capture the statistical tendencies of natural language. This makes PCFGs especially effective for modeling linguistic structures and managing syntactic ambiguity.

### Importance of PCFG in NLP

In NLP, PCFGs are instrumental for several reasons:
- **Handling Ambiguity:** They help in dealing with ambiguities inherent in natural languages by preferring more likely structures over less likely ones.
- **Improving Parsing Accuracy:** PCFGs enhance the accuracy of parsing algorithms, making them more effective in understanding complex sentences.
- **Application in Various NLP Tasks:** They are fundamental in tasks such as machine translation, speech recognition, and syntactic analysis.

### Steps to Induce a PCFG from Productions:

1. Define the Start Symbol:
I identify the start symbol used in the syntax trees (e.g., by reviewing the earlier tree visualization) and create a nonterminal using `nltk.Nonterminal`.

2. Induce the PCFG:
Using the `induce_pcfg` function from NLTK, I generate the PCFG by passing in the start symbol and the list of extracted `productions`. The resulting grammar is stored in a variable named grammar.

3. Display Sample Grammar Rules:
To examine the structure of the induced grammar, I print out the first 10 grammar rules as a sample.

In [7]:
nonterminal_symbol = nltk.Nonterminal('TOP')

In [8]:
grammar = nltk.induce_pcfg(nonterminal_symbol, productions)

In [9]:
grammar.productions()

[TOP -> S_VP PUNC [0.208955],
 S_VP -> VB NP [0.122302],
 VB -> 'List' [0.0180723],
 NP -> NP NP* [0.115942],
 NP -> DT NNS [0.0855072],
 DT -> 'the' [0.626263],
 NNS -> 'flights' [0.746411],
 NP* -> PP NP* [0.121076],
 PP -> IN NP_NNP [0.349927],
 IN -> 'from' [0.452282],
 NP_NNP -> 'Baltimore' [0.0421286],
 NP* -> PP SBAR [0.0112108],
 PP -> TO NP_NNP [0.232796],
 TO -> 'to' [1.0],
 NP_NNP -> 'Seattle' [0.0133038],
 SBAR -> WHNP_WDT S_VP [0.909091],
 WHNP_WDT -> 'that' [0.428571],
 S_VP -> VBP PP [0.0143885],
 VBP -> 'stop' [0.0350877],
 IN -> 'in' [0.10166],
 NP_NNP -> 'Minneapolis' [0.0221729],
 PUNC -> '.' [0.73774],
 TOP -> SQ PUNC [0.0447761],
 SQ -> VBZ SQ* [0.173333],
 VBZ -> 'Does' [0.12069],
 SQ* -> NP VP [0.132353],
 NP -> DT NN [0.101449],
 DT -> 'this' [0.0252525],
 NN -> 'flight' [0.266904],
 VP -> VB NP_NN [0.044586],
 VB -> 'serve' [0.0421687],
 NP_NN -> 'dinner' [0.27451],
 PUNC -> '?' [0.26226],
 TOP -> S PUNC [0.115139],
 S -> NP_PRP VP [0.590164],
 NP_PRP -> 'I' [0

## 05 - Import Sentences to parse

Next, I create a list named sentences by reading all the lines from the dev.strings file. Each line in the file corresponds to a single sentence, and these are stored as individual elements in the list.

This prepares the development data for parsing by organizing the raw sentences into a structured list.

In [10]:
with open('dev.strings') as file:
    sentences = file.read()
    sentences = sentences.split('\n')

sentences

['The flight should be eleven a.m tomorrow .',
 'I would like it to have a stop in New York and I would like a flight that serves breakfast .',
 'Which of these serve dinner ?',
 'Which ones stop in Nashville ?',
 'Are there any flights arriving after eleven a.m ?',
 'What flights do you have from Ontario ?',
 'Where does this flight stop ?',
 "What type of aircraft is Alaska 's flight two eighty two ?",
 'Does the other American flight ?',
 "I 'd like to fly from Tampa to Montreal .",
 "On Tuesday I 'd like to fly from Detroit to Saint Petersburg .",
 'Flight from Long Beach to Columbus on twenty seventh .',
 'Which ones leave in the morning ?',
 'Which of these flights serve dinner ?',
 'Which is last ?',
 'Show me all the fares from one way fares from Tacoma to Montreal .',
 'Which flights serve breakfast ?',
 'What airline provides only connecting flights between Denver and San Francisco ?',
 'List all flights from Burbank to Denver .',
 'Are there any stopovers for Delta one seven

## 06 - CKY Parsing with Unknown Word Handling

### Objective

I then implemented a function named `cky_parsing` to apply the CKY algorithm for sentence parsing using a given Probabilistic Context-Free Grammar (PCFG). This function uses the Viterbi parsing algorithm and accounts for unknown words by replacing them with `<unk>`, ensuring compatibility with the grammar's productions.

### Key Steps in the Implementation:

1. Function Definition:
The function cky_parsing takes two arguments:
    - `sentences`: a list of raw sentence strings to parse
    - `grammar`: the PCFG used by the parser

2. Preprocessing Known Words:

Inside the function, I construct a set of known words by scanning all terminal productions in the grammar. This set is used to identify which words in a sentence are covered by the grammar. Any word not found in this set is replaced with `<unk>` during parsing.

3. Initializing the Viterbi Parser:

I initialize a ViterbiParser using the NLTK library, passing the PCFG as the grammar input.

4. Parsing Sentences:

For each sentence in the list:

    - The sentence is tokenized (split by whitespace).
    - Unknown words are replaced with `<unk>`.
    - The parser attempts to generate all possible parses using parse_all().
    - The most probable parse is selected from the results. If no parse is found, the output reflects that appropriately.

5. Return Value:

    - The function returns a list of tuples, one for each sentence. Each tuple contains:
    - The index of the sentence
    - The original sentence string
    - The best parse tree found, or an appropriate fallback value if parsing fails


In [11]:
def cky_parsing(sentences, grammar):
    import re
    string = str(grammar.productions())
    production_set = set(re.findall(r"'([A-Za-z0-9_\./\\-]*)'", string))
    
    tokenized = []
    for sentence in sentences:
        words = sentence.split()
        tokenized.append(words)

    new_tokenized = []
    for line in tokenized:
        new_words = []
        for word in line:
            if word not in production_set:
                new_word = '<unk>'
                new_words.append(new_word)
            else:
                new_words.append(word)
        new_tokenized.append(new_words)
    
    
    parser = ViterbiParser(grammar)
    
    parsed = []
    for line in new_tokenized:
        parse = parser.parse_all(line)
        parsed.append(parse)
    
    tuples = []
    for i in range(len(sentences)):
        try:
            tuple = (i, sentences[i], parsed[i][0])
            tuples.append(tuple)
        except:
            tuple = (i, sentences[i], parsed[i])
            tuples.append(tuple)
    
    return tuples
    

## 07 - Executing CKY Parsing Function on Sentences

Now parse and print the results.

In [12]:
cky_charts = cky_parsing(sentences, grammar)

for index, sentence, chart in cky_charts:
    print(f"Line {index}: {sentence}")

    if chart:
        print(f"Probability: {math.log(chart.prob())}")  # Using log probability
        print("Parse:")
        chart.pretty_print()  # Pretty print the parse tree      
    else:
        print("No parse available for this sentence")
    print("-------------------------------------------------")


Line 0: The flight should be eleven a.m tomorrow .
No parse available for this sentence
-------------------------------------------------
Line 1: I would like it to have a stop in New York and I would like a flight that serves breakfast .
Probability: -85.26919694246591
Parse:
                                                           TOP                                                                        
               _____________________________________________|______________________________________________________________________    
              S                                                                                                                    |  
   ___________|_______________                                                                                                     |   
  |                           VP                                                                                                   |  
  |       ____________________|______________

### 08 - Export Grammar

To preserve the grammar for future use or analysis, I export the induced grammar using Python’s `pickle` library. This serializes the grammar object and saves it to a file within the working directory of the notebook.

In [13]:
# Importing dill library for model serialization
import dill

# Serialization with dill
with open('grammar.dill', 'wb') as file:
    dill.dump(grammar , file)